In [111]:
example_data = ["[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}",
                "[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}",
                "[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"]
with open("input.txt", "r") as f:
    data = f.read().splitlines()

In [112]:

def parse_data(data):
    parsed = []
    for line in data:
        parts = line.split()
        pattern = parts[0][1:-1]
        # Convert pattern to a bitstring
        target = ''.join('1' if c == '#' else '0' for c in pattern)
        groups = [tuple(map(int, g[1:-1].split(','))) for g in parts[1:-1]]
        for i, g in enumerate(groups):
            g_pattern = 0
            for idx in g:
                g_pattern |= 1 << (idx)
            # Reverse the bit pattern to match the original pattern's order
            g_pattern = '{:0{width}b}'.format(g_pattern, width=len(pattern))[::-1]
            groups[i] = (g_pattern, g)
        numbers = list(map(int, parts[-1][1:-1].split(',')))
        parsed.append((target, groups, numbers))
    return parsed

In [113]:
data_parsed = parse_data(data)

## Part 1

In [114]:
from itertools import combinations_with_replacement
total_presses = []
for line in data_parsed:
    target_pattern, groups, _ = line
    found_solution = False
    if target_pattern in [g[0] for g in groups]:
        total_presses.append(1) # found in one press
        continue
    presses = 2
    while not found_solution:
        product_set = combinations_with_replacement(range(len(groups)), presses)
        for prod in product_set:
            pattern = 0
            for idx in prod:
                group_pattern, _ = groups[idx]
                pattern ^= int(group_pattern, 2)
            if pattern == int(target_pattern, 2):
                #print(f"Found solution for target {target_pattern} with {presses} presses: {prod}")
                found_solution = True
                total_presses.append(presses)
                break
        presses += 1

#print("Presses per line:", total_presses)

print("Total presses:", sum(total_presses))
    

Total presses: 434


## Part 2

In [115]:
import numpy as np
from scipy.optimize import linprog
total_presses = []
for idx, line in enumerate(data_parsed):
    target_pattern, groups, target_joltage = line
    target_joltage = np.array(target_joltage)
    c = np.ones(len(groups))  # Minimize the number of presses
    A_eq = []
    b_eq = target_joltage
    for i in range(len(groups)):
        group_pattern, btn = groups[i]
        btn = np.array(btn)
        row = np.zeros(len(target_joltage))
        btn = np.array(btn)
        row[btn] = 1
        A_eq.append(row)
    A_eq = np.array(A_eq).T
    res = linprog(c, A_eq=A_eq, b_eq=b_eq, integrality=1)
    presses = round(res.fun)
    total_presses.append(presses)
    



In [116]:
print("Total presses with joltage check:", sum(total_presses))
print("Presses per line with joltage check:", total_presses)

Total presses with joltage check: 15132
Presses per line with joltage check: [54, 30, 164, 196, 48, 75, 63, 94, 96, 89, 211, 93, 119, 56, 100, 151, 215, 28, 57, 55, 143, 84, 65, 53, 48, 117, 195, 68, 45, 254, 31, 86, 141, 18, 132, 135, 190, 22, 17, 226, 240, 80, 237, 58, 105, 55, 190, 113, 48, 72, 126, 39, 64, 47, 226, 66, 65, 234, 89, 74, 56, 26, 16, 44, 74, 76, 71, 80, 263, 48, 118, 94, 233, 198, 85, 85, 78, 55, 107, 150, 93, 45, 63, 16, 188, 183, 134, 277, 236, 112, 114, 112, 219, 55, 57, 106, 57, 82, 247, 19, 105, 37, 60, 18, 82, 162, 36, 27, 74, 29, 80, 79, 69, 90, 91, 152, 44, 229, 43, 42, 33, 66, 34, 91, 122, 86, 80, 29, 176, 20, 90, 93, 117, 81, 36, 108, 107, 137, 252, 98, 115, 47, 76, 63, 25, 78, 64, 76, 107, 35, 47, 251, 79]
